# CNN Variations

- [2D Convolutions](#2D-Convolutions)
- [3D Convolutions](#3D-Convolutions)
- [Performance vs Dataset Size](#Performance-vs-Dataset-Size)

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
#import libraries and some constants

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm
import pandas as pd
import uproot as ur
import pickle
from pathlib import Path
import atlas_mpl_style as ampl
ampl.use_atlas_style()

path_prefix = '/AL/Phd/maxml/caloml-atlas/'
plotpath = path_prefix+'classifier/Plots_var/'
modelpath = path_prefix+'classifier/Models/'

# import our resolution utilities
import sys
sys.path.append(path_prefix)
from  util import resolution_util as ru
from  util import plot_util as pu
from  util import ml_util as mu

inputpath = path_prefix+'inputs/'
rootfiles = ["pi0", "piplus", "piminus"]

trees, pdata = mu.setupPionData(inputpath, rootfiles)

In [ ]:
np0 = len(pdata['pi0'])
npp = len(pdata['piplus'])
npm = len(pdata['piminus'])

print("Number of pi0 events: {}".format(np0))
print("Number of pi+ events: {}".format(npp))
print("Number of pi- events: {}".format(npm))
print("Total: {}".format(np0+npp+npm))

pcells = {
    ifile : {
        layer : mu.setupCells(itree, layer, flatten = False)
        for layer in mu.cell_meta
    }
    for ifile, itree in trees.items()
}

In [ ]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import Convolution3D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import MaxPool3D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow import keras as keras

import tensorflow as tf
from sklearn.metrics import roc_curve, auc

gpu_list = ["/gpu:0"]
strategy = tf.distribute.MirroredStrategy(devices=gpu_list)
# strategy = tf.distribute.MirroredStrategy()
ngpu = strategy.num_replicas_in_sync
print ('Number of devices: {}'.format(ngpu))

In [ ]:
training_classes = ['pi0','piplus']
# create train/validation/test subsets containing 70%/10%/20%
# of events from each type of pion event
pdata_merged, pcells_merged, plabels = mu.createTrainingDatasets(training_classes, pdata, pcells)

pcells_merged_reshaped = mu.reshapeSeparateCNN(pcells_merged)
pcells_EMB2_channels = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (16, 16)))

## 2D Convolutions
<div style="text-align: right"> <a href="#CNN-Variations">Top</a> </div>

In [ ]:
def test_model_2d(model,filename='',
               epochs=100):
    '''
    Convenience function for training and calculating
    the area-under-curve for a model.
    '''
    
    # check if model has been trained already
    if Path(modelpath+filename+'.h5').is_file():
        # load model
        model = tf.keras.models.load_model(modelpath+filename+".h5")
    else:
        # train model
        f_history = model.fit(pcells_EMB2_channels[pdata_merged.train],
                            plabels[pdata_merged.train], 
                            validation_data=(pcells_EMB2_channels[pdata_merged.val],
                                             plabels[pdata_merged.val]),
                            epochs=epochs, batch_size=200*ngpu, verbose=2)
        f_history = f_history.history

        # save trained weights and history
        if(filename != ''):
            model.save(modelpath+filename+".h5")
            with open(modelpath+filename +".history",'wb') as model_history_file:
                pickle.dump(f_history, model_history_file)
            
    # get network scores for the dataset
    f_score = model.predict(
        pcells_EMB2_channels
    )
    
    # calculate roc and auc
    f_roc_fpr, f_roc_tpr, f_roc_thresh = roc_curve(
        plabels[pdata_merged.test][:,1],
        f_score[pdata_merged.test,1],
        drop_intermediate=False,
    )
    f_roc_auc = auc(f_roc_fpr, f_roc_tpr)
    
    pu.roc_plot([f_roc_fpr], [f_roc_tpr],
            figfile=plotpath+filename+'_roc.pdf',
            labels=['area = {:.3f}'.format(f_roc_auc)],
            extra_lines=[[[0, 1], [0, 1]]],
            title=filename.replace('_','-')+' ROC curve: classification of $\pi^+$ vs. $\pi^0$')
    return f_roc_fpr, f_roc_tpr

In [ ]:
def cnn_model_nxn(n):
    with strategy.scope():
        # Here come the 16x16 inputs
        input1 = Input(shape=(6, 16, 16), name='input1')
        x = Convolution2D(32, (n, n), activation='relu', data_format = 'channels_first')(input1)
        x = MaxPool2D(pool_size=(2, 2))(x)
        x = Dropout(0.2)(x)
        x = Flatten()(x)
        x = Dense(128, activation='relu')(x)
        x = Dense(64, activation='relu')(x)
        output = Dense(2, activation='softmax')(x)
        model = Model(inputs = [input1], outputs = [output])
        
        # compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
        return model

In [ ]:
# test models
base_filename = 'conv2d_'

res_fpr = []
res_tpr = []
res_auc = []
for i in range(1,4+1):
    fpr, tpr = test_model_2d(cnn_model_nxn(i),filename=base_filename+str(i)+'x'+str(i))
    res_fpr.append(fpr)
    res_tpr.append(tpr)
    res_auc.append( auc(fpr, tpr) )
    
pu.roc_plot(res_fpr, res_tpr,
            figfile=plotpath+'nxn_roc.pdf',
            labels=['area({}) = {:.3f}'.format(str(i+1)+'x'+str(i+1),auc) for i,auc in enumerate(res_auc)],
            extra_lines=[[[0, 1], [0, 1]]],
            title='ROC curve: classification of $\pi^+$ vs. $\pi^0$')


## 3D Convolutions
<div style="text-align: right"> <a href="#CNN-Variations">Top</a> </div>

In [ ]:
def cnn_model_3d():
    with strategy.scope():
        input1 = Input(shape=(1, 6, 16, 16), name='input1')
        x = Convolution3D(32, (2,2,2), activation='relu', data_format = 'channels_first')(input1)
        x = MaxPool3D(pool_size=(2, 2, 2))(x)
        x = Dropout(0.2)(x)
        x = Flatten()(x)
        x = Dense(128, activation='relu')(x)
        x = Dense(64, activation='relu')(x)
        output = Dense(2, activation='softmax')(x)
        model = Model(inputs = [input1], outputs = [output])
        
        # compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
        return model

In [ ]:
model = cnn_model_3d()
history = model.fit(np.expand_dims(pcells_EMB2_channels[pdata_merged.train],axis=1),
                    plabels[pdata_merged.train], 
                    validation_data=(np.expand_dims(pcells_EMB2_channels[pdata_merged.val],axis=1),
                    plabels[pdata_merged.val]),
                    epochs=100, batch_size=200*ngpu, verbose=2)

In [ ]:
scores = model.predict(
    np.expand_dims(pcells_EMB2_channels,axis=1)
)

In [ ]:
roc_fpr, roc_tpr, roc_thresh = roc_curve(
    plabels[pdata_merged.test][:,1],
    scores[pdata_merged.test,1],
    drop_intermediate=False,
)
roc_auc = auc(roc_fpr, roc_tpr)
pu.roc_plot([roc_fpr], [roc_tpr],
    figfile=plotpath+'conv3d_roc.pdf',
    labels=['area(conv3d) = {:.3f}'.format(roc_auc)],
    extra_lines=[[[0, 1], [0, 1]]],
    title='ROC curve: classification of $\pi^+$ vs. $\pi^0$')

In [ ]:
def cnn_model_3d_depth(n):
    with strategy.scope():
        input1 = Input(shape=(1, 6, 16, 16), name='input1')
        x = Convolution3D(32, (n,2,2), activation='relu', data_format = 'channels_first')(input1)
        x = MaxPool3D(pool_size=(2, 2, 2))(x)
        x = Dropout(0.2)(x)
        x = Flatten()(x)
        x = Dense(128, activation='relu')(x)
        x = Dense(64, activation='relu')(x)
        output = Dense(2, activation='softmax')(x)
        model = Model(inputs = [input1], outputs = [output])
        
        # compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
        return model

In [ ]:
depths = [3,4]
model_depth = [cnn_model_3d_depth(n) for n in depths]

In [ ]:
for d,m in zip(depths,model_depth):
    history = m.fit(np.expand_dims(pcells_EMB2_channels[pdata_merged.train],axis=1),
                    plabels[pdata_merged.train], 
                    validation_data=(np.expand_dims(pcells_EMB2_channels[pdata_merged.val],axis=1),
                    plabels[pdata_merged.val]),
                    epochs=100, batch_size=200*ngpu, verbose=2)
    m.save(modelpath+"conv3d_depth_"+str(d)+".h5")
    with open(modelpath+"conv3d_depth_"+str(d)+".history",'wb') as model_history_file:
        pickle.dump(history.history, model_history_file)

In [ ]:
depth_scores = []
for m in model_depth:
    depth_scores.append(m.predict(
        np.expand_dims(pcells_EMB2_channels,axis=1)
    ))

In [ ]:
depth_fpr = []
depth_tpr = []
depth_auc = []
for s in depth_scores:
    roc_fpr, roc_tpr, roc_thresh = roc_curve(
        plabels[pdata_merged.test][:,1],
        s[pdata_merged.test,1],
        drop_intermediate=False,
    )
    depth_fpr.append(roc_fpr)
    depth_tpr.append(roc_tpr)
    depth_auc.append(auc(roc_fpr, roc_tpr))

pu.roc_plot(depth_fpr, depth_tpr,
    figfile=plotpath+'conv3d_depth_roc.pdf',
    labels=['area({}x2x2) = {:.3f}'.format(str(d),auc) for d,auc in zip(depths,depth_auc)],
    extra_lines=[[[0, 1], [0, 1]]],
    title='ROC curve: classification of $\pi^+$ vs. $\pi^0$')

## Performance vs Dataset Size
<div style="text-align: right"> <a href="#CNN-Variations">Top</a> </div>

In [ ]:
def test_model_2d_dataset_size(model,size,filename='',
               epochs=100):
    '''
    Convenience function for training and calculating
    the area-under-curve for a model using a fraction
    of the full dataset.
    '''
    
    rng = np.random.default_rng()
    train_max = int(size*len(pcells_EMB2_channels[pdata_merged.train]))
    train_index = rng.choice(len(pcells_EMB2_channels[pdata_merged.train]),size=train_max,replace=False)
    
    val_max = int(size*len(pcells_EMB2_channels[pdata_merged.val]))
    val_index = rng.choice(len(pcells_EMB2_channels[pdata_merged.val]),size=val_max,replace=False)
    
    # check if model has been trained already
    if Path(modelpath+filename+'.h5').is_file():
        # load model
        model = tf.keras.models.load_model(modelpath+filename+".h5")
    else:
        # train model
        f_history = model.fit(pcells_EMB2_channels[pdata_merged.train][train_index],
                            plabels[pdata_merged.train][train_index], 
                            validation_data=(pcells_EMB2_channels[pdata_merged.val][val_index],
                                             plabels[pdata_merged.val][val_index]),
                            epochs=epochs, batch_size=200*ngpu, verbose=2)
        f_history = f_history.history

        # save trained weights and history
        if(filename != ''):
            model.save(modelpath+filename+".h5")
            with open(modelpath+filename +".history",'wb') as model_history_file:
                pickle.dump(f_history, model_history_file)
            
    # get network scores for the dataset
    f_score = model.predict(
        pcells_EMB2_channels
    )
    
    # calculate roc and auc
    f_roc_fpr, f_roc_tpr, f_roc_thresh = roc_curve(
        plabels[pdata_merged.test][:,1],
        f_score[pdata_merged.test,1],
        drop_intermediate=False,
    )
    f_roc_auc = auc(f_roc_fpr, f_roc_tpr)
    
    pu.roc_plot([f_roc_fpr], [f_roc_tpr],
            figfile=plotpath+filename+'_roc.pdf',
            labels=['area = {:.3f}'.format(f_roc_auc)],
            extra_lines=[[[0, 1], [0, 1]]],
            title=filename.replace('_','-')+' ROC curve: classification of $\pi^+$ vs. $\pi^0$')
    return f_roc_fpr, f_roc_tpr

In [ ]:
def cnn_model_baseline():
    with strategy.scope():
        input1 = Input(shape=(6, 16, 16), name='input1')
        x = Convolution2D(32, (2,2), activation='relu', data_format = 'channels_first')(input1)
        x = MaxPool2D(pool_size=(2, 2))(x)
        x = Dropout(0.2)(x)
        x = Flatten()(x)
        x = Dense(128, activation='relu')(x)
        x = Dense(64, activation='relu')(x)
        output = Dense(2, activation='softmax')(x)
        model = Model(inputs = [input1], outputs = [output])
        
        # compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
        return model

In [ ]:
sizes = [0.2,0.4,0.6,0.8,1.0]
size_fpr = []
size_tpr = []
size_auc = []
for size in sizes:
    fpr, tpr = test_model_2d_dataset_size(cnn_model_baseline(),size,
                                         filename='dsetsize_'+str(size).replace('.',''))
    size_fpr.append(fpr)
    size_tpr.append(tpr)
    size_auc.append(auc(fpr,tpr))
    
pu.roc_plot(size_fpr, size_tpr,
    figfile=plotpath+'dsetsize_roc.pdf',
    labels=['area({}) = {:.3f}'.format(str(s),auc) for s,auc in zip(sizes,size_auc)],
    extra_lines=[[[0, 1], [0, 1]]],
    title='ROC curve: classification of $\pi^+$ vs. $\pi^0$')

In [ ]:
epochs = [20,40,60,80,100,150,200]
epoch_fpr = []
epoch_tpr = []
epoch_auc = []
for e in epochs:
    fpr, tpr = test_model_2d_dataset_size(cnn_model_baseline(),1.0,
                                         filename='epochsize_'+str(e),
                                         epochs=e)
    epoch_tpr.append(tpr)
    epoch_fpr.append(fpr)
    epoch_auc.append(auc(fpr,tpr))
    
pu.roc_plot(epoch_fpr, epoch_tpr,
            figfile=plotpath+'epochsize_roc.pdf',
            labels=['area({}) = {:.3f}'.format(str(e),auc) for e,auc in zip(epochs,epoch_auc)],
            extra_lines=[[[0, 1], [0, 1]]],
            title='ROC curves over epochs: classification of $\pi^+$ vs. $\pi^0$')